In [1]:
output_dir = "/home/jovyan/20-restaurant-sales/data/out"
preprocessed_path = f'{output_dir}/preprocessed'

# Function to retrieve data from kaggle for the train component

In [2]:
def get_data(download_url, out_path):
    import subprocess
    import pickle
    import sys
    import subprocess
    
    
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas'])
    
    import pandas as pd
    
    
    default_url = "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/4272/33759/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1602334426&Signature=H8tIQw1fzN0gce0au2F4kTPWaR9zaFpa0n6PcXOMWcFxs0LW%2Fvsl8aMzspOHE%2FV8J7chahxFJzHHLccOhG%2BY8F0m12vZqP%2FeR2o1ZFlr%2BL9VrwvXzndtJ53Q4ZQCFtj3wP9qSw7FAdPROiv3k95sd4moUaPKlyxdxih5dMGDyMIrrIkWJwiIHvs3J50nW2xA%2F7QJKv%2B9tEO5GianSsxkH7HxeEzdRf5A8Cz9sZjlNKdKUrpkHQO4ZDQ8x9a97mA9noK62Gp5mPhwuHYa5OIhXgRfLyim0C1zL%2BNJUGWZUbHmPys9kJTG5h%2Bh68OR5DwkYWlfF3eFIU6K6VKsTkG%2F0w%3D%3D&response-content-disposition=attachment%3B+filename%3Drestaurant-revenue-prediction.zip"
    url = download_url if download_url else default_url
    
    subprocess.run(["wget", url, "-O", "restaurant-revenue-prediction.zip"])
    
    subprocess.run(["unzip", "*.zip"])
    subprocess.run(["unzip", "*.zip"])
    print("Files Unzipped....")
    subprocess.call(["rm", "-rf", "*.zip"])
    
    train_data = pd.read_csv("train.csv")
    test_data = pd.read_csv("test.csv")
    
    with open(f"{out_path}/train_data.pkl", "wb") as train:
        pickle.dump(train_data, train)
        
    with open(f"{out_path}/test_data.pkl", "wb") as test:
        pickle.dump(test_data, test)
    

In [3]:
get_data("", output_dir)

Files Unzipped....


# Function to preprocess the data using correlation analysis

In [4]:
def preprocess_corr_analysis(out_path):
    
    import sys
    import subprocess
    
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas', 'scikit-learn'])
#     subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn'])
    
    import pickle
    import numpy as np
    from sklearn.preprocessing import StandardScaler, LabelEncoder
    from sklearn.model_selection import train_test_split
    import pandas as pd
    
    with open(f"{out_path}/train_data.pkl", 'rb') as f:
        train = pickle.load(f)
        
    with open(f"{out_path}/test_data.pkl", 'rb') as f:
        test = pickle.load(f) 
       
    train.drop(['Id', 'Open Date', 'City'], axis=1, inplace=True)
    test.drop(['Id', 'Open Date', 'City'], axis=1, inplace=True)
    
    city_group_encoder = LabelEncoder()
    scaler = StandardScaler()

    train['City Group'] = city_group_encoder.fit_transform(train['City Group'])
    test['City Group'] = city_group_encoder.transform(test['City Group'])

    train = pd.get_dummies(train, drop_first=True)
    test = pd.get_dummies(test, drop_first=True)

    X = train.drop(['revenue'], axis=1)
    y = train[['revenue']]

    test.drop(["Type_MB"], axis=1, inplace=True)
    
    corr_matrix = X.corr().abs()
    
    condition = np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool)
    upper = corr_matrix.where(condition)
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    
    X.drop(to_drop, axis=1, inplace=True)
    test.drop(to_drop, axis=1, inplace=True)
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)

    X_train = scaler.fit_transform(X_train)

    X_val = scaler.transform(X_val)

    test = scaler.transform(test)
    
#     outfile = TemporaryFile()
    np.savez_compressed(f'{out_path}/corr-analysis-data.npz',
                       X_train=X_train,
                       X_val=X_val,
                       y_train=y_train,
                       y_val=y_val,
                       test_data=test)
    print("Preprocessing Complete")

In [5]:
preprocess_corr_analysis(output_dir)

Preprocessing Complete


# Function to train the data with Linear Regression Model with Ridge regularization - train component (train-ridge)

In [6]:
def train_corr_ridge(out_path, ridge_model_file):
    
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas', 'scikit-learn'])
    from sklearn.linear_model import Ridge
    from sklearn.metrics import mean_squared_error
    import numpy as np
    import pickle
    
    preprocessed_data = np.load(f'{out_path}/corr-analysis-data.npz')
    X_train = preprocessed_data['X_train']
    y_train = preprocessed_data['y_train']
    
    X_val = preprocessed_data['X_val']
    y_val = preprocessed_data['y_val']
    ridge = Ridge(alpha=0.1)
    ridge.fit(X_train, y_train)
    
    y_pred = ridge.predict(X_val)
    
    with open(f'{out_path}/{ridge_model_file}', "wb") as model:
        pickle.dump(ridge, model)
    
    RMSE = np.sqrt(mean_squared_error(y_pred, y_val))
    
    print(f"RMSE: {RMSE}")

    
    
    
    

In [7]:
train_corr_ridge(output_dir, 'ridge-corr-model.pkl')

RMSE: 4261584.754762309


# Function to predict with Linear Regression Model with Ridge regularization - train component (predict-ridge)

In [8]:
def predict_ridge(out_path, ridge_model_file):
    
    import pickle
    import sys
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas', 'scikit-learn'])
    import numpy as np
    
    with open(f'{out_path}/{ridge_model_file}', "rb") as model:
        ridge = pickle.load(model)
        
    test_data = np.load(f'{out_path}/corr-analysis-data.npz')['test_data']
    
    predictions = ridge.predict(test_data)
    
    np.savetxt(f'{out_path}/ridge-corr.txt', predictions, fmt='%1.2f')
    
#     sample_submission = pd.read_csv()
    print("File saved as ridge-corr.txt")

In [9]:
predict_ridge(output_dir, 'corr-ridge-model.pkl')

File saved as ridge-corr.txt


In [10]:
# 

In [11]:
import kfp
from kfp import dsl
import kfp.components as comp

In [12]:
download_op = comp.func_to_container_op(get_data, base_image="python:3.7")
preprocess_op = comp.func_to_container_op(preprocess_corr_analysis, base_image="tensorflow/tensorflow:latest-gpu-py3")
train_corr_ridge_op = comp.func_to_container_op(train_corr_ridge, base_image="tensorflow/tensorflow:latest-gpu-py3")
predict_ridge_op = comp.func_to_container_op(predict_ridge, base_image="tensorflow/tensorflow:latest-gpu-py3")

In [13]:
@dsl.pipeline(
    name="Restaurant Sales Revenue Prediction",
    description="A Machine Learning Pipeline for Revenue prediction model"
)

def revenue_prediction_pipeline(
    out_path: str,
    ridge_corr_file: str,
    download_url: str
):
    volume_op = dsl.VolumeOp(
        name="create_volume",
        resource_name="data-volume",
        size="1Gi",
        modes=dsl.VOLUME_MODE_RWO)
    
    download_container = download_op(download_url, out_path).add_pvolumes({out_path: volume_op.volume})
    
    revenue_preprocess_container = preprocess_op(out_path).add_pvolumes({out_path: download_container.pvolume})
    
    train_ridge_container = train_corr_ridge_op(out_path, ridge_corr_file).add_pvolumes({out_path: revenue_preprocess_container.pvolume})
    
    predict_ridge_container = predict_ridge_op(out_path, ridge_corr_file) \
                                        .add_pvolumes({out_path: train_ridge_container.pvolume})
    
    revenue_result_container = dsl.ContainerOp(
        name="predictions",
        image="library/bash:4.4.23",
        pvolumes={out_path: predict_ridge_container.pvolume},
        arguments=['cat', f'{out_path}/ridge-corr.txt']
    )

In [14]:
OUT_PATH = '/mnt'
RIDGE_CORR_MODEL_PATH = 'ridge-corr-model.pkl'
DOWNLOAD_URL = ""

In [15]:
pipeline_func = revenue_prediction_pipeline

In [16]:
experiment_name = 'kubeflow_revenue_prediction'
run_name = f'{pipeline_func.__name__} run'
arguments = {
    "out_path": OUT_PATH,
    "ridge_corr_file": RIDGE_CORR_MODEL_PATH,
    "download_url": DOWNLOAD_URL
}

kfp.compiler.Compiler().compile(pipeline_func, f'{experiment_name}.zip')



In [17]:
client = kfp.Client()

In [18]:
run_result = client.create_run_from_pipeline_func(pipeline_func,
                                                 experiment_name=experiment_name,
                                                 run_name=run_name,
                                                 arguments=arguments
                                                 )